In [319]:
import pandas as pd
import numpy as np

# Reading the Movie Rating dataset

In [320]:
#Reading the file:
movie_movie_datasetdataset=pd.read_csv("movie_ratings.csv")
#Dropping user name column:

movie_ratings=movie_dataset.drop('users', axis=1)
print(movie_ratings.head(5))
print(" Shape of movie matrix is :", movie_ratings.shape)

   movie1  movie2  movie3  movie4  movie5  movie6  movie7  movie8  movie9  \
0     3.0     5.0     1.0     NaN     NaN     2.0     4.0     NaN     1.0   
1     1.0     3.0     1.0     4.0     NaN     2.0     NaN     1.0     3.0   
2     2.0     3.0     NaN     4.0     3.0     NaN     1.0     NaN     3.0   
3     1.0     NaN     4.0     NaN     5.0     2.0     3.0     1.0     5.0   
4     NaN     1.0     2.0     5.0     NaN     1.0     3.0     NaN     3.0   

   movie10  
0      4.0  
1      3.0  
2      5.0  
3      NaN  
4      1.0  
 Shape of movie matrix is : (50, 10)


# Content based Matrix factorization using regularization:

In [321]:
# Function for matrix factorization:

def matrix_factorization(R, P, Q, K, steps=10000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        #print(e)
        if e < 100:
            break
    return P, Q.T,e

# Defining all the parameters and calling the function:

In [338]:
#Defining all the parameters:

R = np.array(movie_ratings)
N = len(R)
M = len(R[0])
K = 3
P = np.random.rand(N,K)
#print(P.shape)
Q = np.random.rand(M,K) 
#print(Q.shape)

#Calling the function:

nP, nQ, e= matrix_factorization(R, P, Q, K)
print("Breakout Error form the function is", e)

#Combining it again as a full matrix
nR = np.dot(nP, nQ.T)
print(nR)

Breakout Error form the function is 251.4256118526567
[[ 2.22192713  4.5777817   0.78931214  4.03116789  1.85843455  2.22458062
   4.16829405  3.06822826  2.89385696  2.92793208]
 [ 1.73827448  3.17278756  1.68927892  3.08609757  1.39666861  1.92571228
   2.2041418   1.65604361  2.68816554  2.84329982]
 [ 2.02688555  2.66686176  4.44843002  4.16019821  2.73700666  2.42040161
   1.18449936  0.98011079  3.85839917  4.29233802]
 [ 2.09529223  2.60542661  4.68573897  5.26072839  4.49339027  2.17595321
   2.56200303  1.98878799  3.67726631  4.07702501]
 [ 1.25064579  1.49199944  2.60458464  4.26653804  4.61800928  0.87176643
   3.36155705  2.51634001  1.71966866  1.86682558]
 [ 1.97304033  3.23650849  2.64548645  4.16982862  2.77827429  2.05981615
   2.83478095  2.13899779  3.08407267  3.30367409]
 [ 1.62661824  2.02213974  3.55131906  4.36235097  3.96450427  1.57787817
   2.47634107  1.89670533  2.72282895  3.00607102]
 [ 2.87296277  5.40587354  2.57849617  4.45552637  1.18176697  3.378532

# Comparing matrices and inserting predicted values in the original matrix :

In [339]:
# Replacing all the movie rating by -1 that are already watched by users:

def eliminating_watched_movies(R):
    for i in range(len(R)):
            for j in range(len(R[i])):
                if not np.isnan(R[i][j]): 
                    R[i][j]=-1               
    return R

In [340]:
# Adding predictions to this matrix:

def Adding_predictions(R,nR):
    #First calling the function eliminating_watched_movies:
    eliminating_watched_movies(R)
    for i in range(len(R)):
            for j in range(len(R[i])):
                if np.isnan(R[i][j]): 
                    R[i][j]=nR[i][j]             
    return R

# Preparing the final matrix:

In [341]:
#Final Matrix of ratings to use for recommendation:

Adding_predictions(R,nR)

array([[-1.        , -1.        , -1.        ,  4.03116789,  1.85843455,
        -1.        , -1.        ,  3.06822826, -1.        , -1.        ],
       [-1.        , -1.        , -1.        , -1.        ,  1.39666861,
        -1.        ,  2.2041418 , -1.        , -1.        , -1.        ],
       [-1.        , -1.        ,  4.44843002, -1.        , -1.        ,
         2.42040161, -1.        ,  0.98011079, -1.        , -1.        ],
       [-1.        ,  2.60542661, -1.        ,  5.26072839, -1.        ,
        -1.        , -1.        , -1.        , -1.        ,  4.07702501],
       [ 1.25064579, -1.        , -1.        , -1.        ,  4.61800928,
        -1.        , -1.        ,  2.51634001, -1.        , -1.        ],
       [-1.        ,  3.23650849,  2.64548645,  4.16982862,  2.77827429,
        -1.        , -1.        , -1.        ,  3.08407267, -1.        ],
       [ 1.62661824, -1.        , -1.        , -1.        ,  3.96450427,
        -1.        , -1.        , -1.        

# Adding column name and indexes::

In [1]:
#Defing a list ranging 1-50 to add 50 users:

mylist = list(range(1,51))
print(mylist)

#Add indexes and column name:
Ratings=pd.DataFrame(R,columns=['movie1','movie2','movie3','movie4','movie5',
                                'movie6','movie7','movie8','movie9','movie10'],index=mylist)
print(Ratings)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]


NameError: name 'pd' is not defined

# Sorting the ratings:

In [343]:
#For each user recommendation ratings are sorted:

def sorted_ratings():
    for i in range(len(mylist)):
        a=Ratings.iloc[i]
        print(a.sort_values(ascending=False))
sorted_ratings()

#We can pick top 2 or top 3 movies to send recommendation to users.

movie4     4.031168
movie8     3.068228
movie5     1.858435
movie10   -1.000000
movie9    -1.000000
movie7    -1.000000
movie6    -1.000000
movie3    -1.000000
movie2    -1.000000
movie1    -1.000000
Name: 1, dtype: float64
movie7     2.204142
movie5     1.396669
movie10   -1.000000
movie9    -1.000000
movie8    -1.000000
movie6    -1.000000
movie4    -1.000000
movie3    -1.000000
movie2    -1.000000
movie1    -1.000000
Name: 2, dtype: float64
movie3     4.448430
movie6     2.420402
movie8     0.980111
movie10   -1.000000
movie9    -1.000000
movie7    -1.000000
movie5    -1.000000
movie4    -1.000000
movie2    -1.000000
movie1    -1.000000
Name: 3, dtype: float64
movie4     5.260728
movie10    4.077025
movie2     2.605427
movie9    -1.000000
movie8    -1.000000
movie7    -1.000000
movie6    -1.000000
movie5    -1.000000
movie3    -1.000000
movie1    -1.000000
Name: 4, dtype: float64
movie5     4.618009
movie8     2.516340
movie1     1.250646
movie10   -1.000000
movie9    -1.000000
movi

# Top 2 recommended movie for a specific user:

In [348]:
def top_2_recommendations_for_user(user_id):
        #Using loc to get index based values:
        a=Ratings.loc[user_id].sort_values(ascending=False)
        if a[0]>0 and a[1]>0: 
            print(a[0:2])
        if a[0]>0 and a[1]<0: 
            print(a[0:1])
        if a[0]<0: 
            print("No Recommendation")
top_2_recommendations_for_user(3)   

#There is no user who has watched all the movies
#Try only one recommendation with movie 48
#Try two recommendation with movie 1

movie3    4.448430
movie6    2.420402
Name: 3, dtype: float64
